# Power Outage Analysis

**Name(s)**: Vivian Lin, Selina Zhang

**Website Link**: https://vivianlinnn.github.io/power_outage_analysis/

In [55]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Introduction

This project revolves around a dataset that provides rich information about power outages across the states, including time, location, climate conditions, and urbanization levels. We can use this dataset to identify patterns of power outages to infer problems of these outages. The main question is **where are the causes of major outages?** Readers of our website should care because we use electricity to make everything easier for our lives such as lights, various machinaries, and the internet. Thus, analyzing power outages can help us better prepare when and where to expect power outages. 

The dataset comprises of 1534 rows.

These are the columns we used:

- `YEAR` - year when the outage event occurred
- `MONTH`- the month when the outage event occurred
- `NERC.REGION` - The North American Electric Reliability Corporation (NERC) regions impacted by outage event
- `CLIMATE.REGION` - U.S. Climate regions as specified by National Centers for Environmental Information
- `CLIMATE.CATEGORY` - This represents the climate episodes corresponding to the years. The categories—“Warm”, “Cold” or “Normal” 
- `CAUSE.CATEGORY` - Categories of all the events causing the major power outages
- `OUTAGE.DURATION` - how long the outage lasted
- `CUSTOMERS.AFFECTED` - Number of customers affected by the power outage
- `NERC.REGION` - The North American Electric Reliability Corporation (NERC) regions involved in the outage event
- `RES.PERCEN`- Percentage of residential electricity consumption compared to the total electricity consumption in the state (in %)
- `ANOMALY.LEVEL` - This represents the oceanic El Niño/La Niña (ONI) index referring to the cold and warm episodes by season. It is estimated as a 3-month running mean of ERSST.v4 SST anomalies in the Niño 3.4 region (5°N to 5°S, 120–170°W)
- `TOTAL.REALGSP` - Real GSP contributed by all industries (total) (measured in 2009 chained U.S. dollars)
- `'DEMAND.LOSS.MW'` - Amount of peak demand lost during an outage event (in Megawatt) [but in many cases, total demand is reported]


## Data Cleaning and Exploratory Data Analysis

In [56]:
data = pd.read_excel("outage.xlsx")
data.head(7)
col_names = data.iloc[4]
data.columns = col_names.values
data = data.iloc[6:].reset_index(drop=True)
data.drop(columns=['variables'], inplace=True)
data.head()
data

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
0,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
1,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
2,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
3,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
4,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06 00:00:00,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1530,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.599649,2.401765,2.401765
1531,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256
1532,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.307744,1.692256,1.692256


In [109]:
columns_to_extract = [
    'YEAR', 
    'MONTH', 
    'NERC.REGION', 
    'CLIMATE.REGION', 
    'CLIMATE.CATEGORY', 
    'CAUSE.CATEGORY', 
    'OUTAGE.DURATION', 
    'CUSTOMERS.AFFECTED', 
    'RES.PERCEN', 
    'ANOMALY.LEVEL', 
    'TOTAL.REALGSP',
    'DEMAND.LOSS.MW'
]
data = data[columns_to_extract]
data.head()

,YEAR,MONTH,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,CUSTOMERS.AFFECTED,RES.PERCEN,ANOMALY.LEVEL,TOTAL.REALGSP,DEMAND.LOSS.MW
0,2011,7,MRO,East North Central,normal,severe weather,3060,70000,35.549073,-0.3,274182,NaN
1,2014,5,MRO,East North Central,normal,intentional attack,1,NaN,30.032487,-0.1,291955,NaN
2,2010,10,MRO,East North Central,cold,severe weather,3000,70000,28.097672,-1.5,267895,NaN
3,2012,6,MRO,East North Central,normal,severe weather,2550,68200,31.994099,-0.1,277627,NaN
4,2015,7,MRO,East North Central,warm,severe weather,1740,250000,33.982576,1.2,292023,250


In [119]:
print(data.head().to_markdown(index=True))


|    |   YEAR |   MONTH | NERC.REGION   | CLIMATE.REGION     | CLIMATE.CATEGORY   | CAUSE.CATEGORY     |   OUTAGE.DURATION |   CUSTOMERS.AFFECTED |   RES.PERCEN |   ANOMALY.LEVEL |   TOTAL.REALGSP |   DEMAND.LOSS.MW |
|---:|-------:|--------:|:--------------|:-------------------|:-------------------|:-------------------|------------------:|---------------------:|-------------:|----------------:|----------------:|-----------------:|
|  0 |   2011 |       7 | MRO           | East North Central | normal             | severe weather     |              3060 |                70000 |      35.5491 |            -0.3 |          274182 |              nan |
|  1 |   2014 |       5 | MRO           | East North Central | normal             | intentional attack |                 1 |                  nan |      30.0325 |            -0.1 |          291955 |              nan |
|  2 |   2010 |      10 | MRO           | East North Central | cold               | severe weather     |              3000 |    

In [118]:
data.head().T

,0,1,2,3,4
YEAR,2011,2014,2010,2012,2015
MONTH,7,5,10,6,7
NERC.REGION,MRO,MRO,MRO,MRO,MRO
CLIMATE.REGION,East North Central,East North Central,East North Central,East North Central,East North Central
CLIMATE.CATEGORY,normal,normal,cold,normal,warm
CAUSE.CATEGORY,severe weather,intentional attack,severe weather,severe weather,severe weather
OUTAGE.DURATION,3060,1,3000,2550,1740
CUSTOMERS.AFFECTED,70000,NaN,70000,68200,250000
RES.PERCEN,35.549073,30.032487,28.097672,31.994099,33.982576
ANOMALY.LEVEL,-0.3,-0.1,-1.5,-0.1,1.2


### Univariate Analyses
#### Climate Regions

In [58]:
# state_pop_means = data
# state_pop_means['STATE_POPULATION_MEAN'] = state_pop_means.groupby('U.S._STATE')['POPULATION'].transform(lambda ser: ser.mean())

# sort_alpha_order = data.sort_values('U.S._STATE')

region_counts = data['CLIMATE.REGION'].value_counts()
new_df = pd.DataFrame(region_counts)
new_df = new_df.reset_index()

fig = px.bar(
    new_df,
    x='CLIMATE.REGION',
    y='count',
    title='Distribution of Outage Counts Amongst Different Climate Regions',
    labels={'CLIMATE.REGION': 'Climate Region'},
    color_discrete_sequence=['#bd3786']
    
)
fig.update_layout(
    yaxis_title = 'Number of Outages'
)
fig.show()

#### Month

In [59]:
# sort_year = data.sort_values('MONTH')
fig = px.histogram(
    data,
    x='MONTH',
    title='Distribution of Outage Counts Amongst Different Months',
    labels={'MONTH': 'Months'},
    color_discrete_sequence=['#d8576b']
)

fig.update_layout(
    yaxis_title='Number of Outages'
)
fig.show()

The histogram shows the distribution of outages that occur every month. Outages are more likely to happen in months with extreme temperatures such as June, July, January, and February.

### Bivariate Analysis
#### Climate Region vs. Outage Duration

In [60]:
def zscore(ser):
    ser_mean = ser.mean()
    ser_std = ser.std(ddof=0)

    return (ser - ser_mean)/ser_std

In [61]:
data_w_zscore = data.copy()
all_zscores = data.groupby('CLIMATE.REGION')['OUTAGE.DURATION'].transform(zscore)
data_w_zscore['zscores'] = all_zscores

min_zscore = data_w_zscore['zscores'].min()
max_zscore = data_w_zscore['zscores'].max()
bounds = min(abs(min_zscore), abs(max_zscore))

no_outlier = data_w_zscore[np.abs(data_w_zscore['zscores']) <= bounds]

fig = px.box(
    no_outlier,
    x=no_outlier['CLIMATE.REGION'],
    y=no_outlier['OUTAGE.DURATION'],
    labels={"CLIMATE.REGION": "Climate Regions", "OUTAGE.DURATION": "Outage Durations"},
    title="Climate Regions vs. Outage Durations",
    color_discrete_sequence=['#ed7953']
)
fig.show()


#### 2nd bivariate

In [62]:
fig = px.scatter(
    data,
    x=data['CAUSE.CATEGORY'],
    y=data['CUSTOMERS.AFFECTED'],
    labels={'CAUSE.CATEGORY': 'Cause Cateogories', 'CUSTOMERS.AFFECTED': 'Number of Customers Affected'},
    title="Cause Category vs. Customers Affected",
    color_discrete_sequence=['#fb9f3a']
)
fig.show()

In [63]:
def assign_range(value):
    if pd.isna(value):
        return np.nan
    if value <= 1e+6:
        return '(0, 1M]'
    elif value <= 2e+6:
        return '(1M, 2M]'
    elif value <= 3e+6:
        return '(2M, 3M]'
    else:
        return '(3M, )'
    
data['customers_affected_ranges'] = data['CUSTOMERS.AFFECTED'].apply(assign_range)
data

/var/folders/r2/plrh1xlx4_b6wmq2hw3_92dw0000gn/T/ipykernel_84582/4902139.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,YEAR,MONTH,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,CUSTOMERS.AFFECTED,RES.PERCEN,ANOMALY.LEVEL,TOTAL.REALGSP,DEMAND.LOSS.MW,customers_affected_ranges
0,2011,7,MRO,East North Central,normal,severe weather,3060,70000,35.549073,-0.3,274182,NaN,"(0, 1M]"
1,2014,5,MRO,East North Central,normal,intentional attack,1,NaN,30.032487,-0.1,291955,NaN,NaN
2,2010,10,MRO,East North Central,cold,severe weather,3000,70000,28.097672,-1.5,267895,NaN,"(0, 1M]"
3,2012,6,MRO,East North Central,normal,severe weather,2550,68200,31.994099,-0.1,277627,NaN,"(0, 1M]"
4,2015,7,MRO,East North Central,warm,severe weather,1740,250000,33.982576,1.2,292023,250,"(0, 1M]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12,MRO,West North Central,cold,public appeal,720,34500,37.212544,-0.9,39067,155,"(0, 1M]"
1530,2006,NaN,MRO,West North Central,NaN,fuel supply emergency,NaN,NaN,NaN,NaN,27868,1650,NaN
1531,2009,8,RFC,West North Central,warm,islanding,59,NaN,36.564432,0.5,36504,84,NaN
1532,2009,8,MRO,West North Central,warm,islanding,181,NaN,36.564432,0.5,36504,373,NaN


#### Interesting Aggregates

In [64]:
pivot_table1 = data.pivot_table(
    index='CLIMATE.REGION',
    columns='customers_affected_ranges',
    values='CUSTOMERS.AFFECTED',
    aggfunc='count',
    fill_value=0,
)
pivot_table1

customers_affected_ranges,"(0, 1M]","(1M, 2M]","(2M, 3M]","(3M, )"
CLIMATE.REGION,,,,
Central,156,1,0,0
East North Central,117,0,1,0
Northeast,265,0,0,1
Northwest,62,0,0,0
South,149,4,2,0
Southeast,140,1,1,1
Southwest,45,0,0,0
West,125,6,1,0
West North Central,7,0,0,0


In [120]:
print(pivot_table1.to_markdown(index=True))


| CLIMATE.REGION     |   (0, 1M] |   (1M, 2M] |   (2M, 3M] |   (3M, ) |
|:-------------------|----------:|-----------:|-----------:|---------:|
| Central            |       156 |          1 |          0 |        0 |
| East North Central |       117 |          0 |          1 |        0 |
| Northeast          |       265 |          0 |          0 |        1 |
| Northwest          |        62 |          0 |          0 |        0 |
| South              |       149 |          4 |          2 |        0 |
| Southeast          |       140 |          1 |          1 |        1 |
| Southwest          |        45 |          0 |          0 |        0 |
| West               |       125 |          6 |          1 |        0 |
| West North Central |         7 |          0 |          0 |        0 |


In [65]:
# Assuming `data` is your DataFrame containing the dataset
# Grouping by NERC.REGION and calculating the mean of OUTAGE.DURATION, CUSTOMERS.AFFECTED, and DEMAND.LOSS.MW
severity_metrics = data.groupby('NERC.REGION').agg({
    'OUTAGE.DURATION': 'mean',
    'CUSTOMERS.AFFECTED': 'mean',
    'DEMAND.LOSS.MW': 'mean'
})

severity_metrics

,OUTAGE.DURATION,CUSTOMERS.AFFECTED,DEMAND.LOSS.MW
NERC.REGION,,,
ASCC,NaN,14273.0,35.0
ECAR,5603.3125,256354.1875,1314.483871
FRCC,4271.116279,289778.181818,804.45
"FRCC, SERC",372.0,NaN,NaN
HECO,895.333333,126728.666667,466.666667
HI,1367.0,294000.0,1060.0
MRO,2933.590909,88984.965517,279.5
NPCC,3262.170068,108726.037037,930.123288
PR,174.0,62000.0,220.0


In [121]:
print(severity_metrics.to_markdown(index=True))

| NERC.REGION   |   OUTAGE.DURATION |   CUSTOMERS.AFFECTED |   DEMAND.LOSS.MW |
|:--------------|------------------:|---------------------:|-----------------:|
| ASCC          |           nan     |                14273 |           35     |
| ECAR          |          5603.31  |               256354 |         1314.48  |
| FRCC          |          4271.12  |               289778 |          804.45  |
| FRCC, SERC    |           372     |                  nan |          nan     |
| HECO          |           895.333 |               126729 |          466.667 |
| HI            |          1367     |               294000 |         1060     |
| MRO           |          2933.59  |                88985 |          279.5   |
| NPCC          |          3262.17  |               108726 |          930.123 |
| PR            |           174     |                62000 |          220     |
| RFC           |          3477.96  |               127894 |          293.153 |
| SERC          |          1737.99  |   

## Step 3: Assessment of Missingness

### CLIMATE.CATEGORY

**Null Hypothesis:** The missingness of `CUSTOMERS.AFFECTED` is independent of the column `CLIMATE.CATEGORY`. 

**Alternative Hypothesis:** The missingness of `CUSTOMERS.AFFECTED` is dependent of the column `CLIMATE.CATEGORY`.

In [66]:
distribution = data.copy()
distribution['is_missing'] = distribution['CUSTOMERS.AFFECTED'].isna()

pivoted1 = distribution.pivot_table(index='CLIMATE.CATEGORY', 
                         columns='is_missing', 
                         aggfunc='size',
                         fill_value=0)

pivoted1

is_missing,False,True
CLIMATE.CATEGORY,,
cold,329,144
normal,525,219
warm,230,78


In [67]:
tot = pivoted1.sum(axis=1)
pivot_copy = pivoted1.copy()
pivot_copy[False] = pivoted1[False]/tot
pivot_copy[True] = pivoted1[True]/tot
pivot_copy

is_missing,False,True
CLIMATE.CATEGORY,,
cold,0.695560,0.304440
normal,0.705645,0.294355
warm,0.746753,0.253247


In [68]:
pivot_copy.plot(kind='barh', barmode='group', title='Distribution of Missing Data Across all Climate Categories')


In [69]:
obs_tvd = pivoted1.diff(axis=1).iloc[:,-1].abs().sum()/ 2
obs_tvd


np.float64(321.5)

In [70]:
n_repetitions = 1000
shuffled = data.copy()
tvds = []
for _ in range(n_repetitions):

    # Shuffling genders. 
    # Note that we are assigning back to the same DataFrame for performance reasons; 
    # see https://dsc80.com/resources/lectures/lec07/lec07-fast-permutation-tests.html.
    shuffled['shuffled'] = np.random.permutation(shuffled['CUSTOMERS.AFFECTED'])

    # Computing and storing the TVD.
    shuffled['is_missing'] = shuffled['shuffled'].isna()

    pivoted = shuffled.pivot_table(index='CLIMATE.CATEGORY', 
                                            columns='is_missing', 
                                            aggfunc='size',
                                            fill_value=0)

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

In [71]:
fig = px.histogram(x=tvds, histnorm='probability', labels={'x': 'TVD'}, title='Empirical Distribution of TVD')
fig.add_vline(
    x=obs_tvd,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Observed TVD = {obs_tvd}"
)
fig.show()


In [72]:
p_val = (np.array(tvds) >= obs_tvd).mean()
p_val

np.float64(0.756)

The observed TVD is 321.5, resulting in the p-value of 0.753, which is greater than our significance level of 0.05. We fail to reject the null hypothesis and conclude that the missingness of CUSTOMERS.AFFECTED is not dependent on the column CLIMATE.CATEGORY and is independent from CLIMATE.CATEGORY.

### CLIMATE.REGION

**Null Hypothesis:** The missingness of `CUSTOMERS.AFFECTED` is independent of the column `CLIMATE.REGION`. 

**Alternative Hypothesis:** The missingness of `CUSTOMERS.AFFECTED` is dependent of the column `CLIMATE.REGION`.

In [73]:
distribution = data.copy()
distribution['is_missing'] = distribution['CUSTOMERS.AFFECTED'].isna()

pivoted2 = distribution.pivot_table(index='CLIMATE.REGION', 
                         columns='is_missing', 
                         aggfunc='size',
                         fill_value=0)
pivoted2

is_missing,False,True
CLIMATE.REGION,,
Central,157,43
East North Central,118,20
Northeast,266,84
Northwest,62,70
South,155,74
Southeast,143,10
Southwest,45,47
West,132,85
West North Central,7,10


In [74]:
tot = pivoted2.sum(axis=1)
pivot_copy = pivoted2.copy()
pivot_copy[False] = pivoted2[False]/tot
pivot_copy[True] = pivoted2[True]/tot
pivot_copy

is_missing,False,True
CLIMATE.REGION,,
Central,0.785000,0.215000
East North Central,0.855072,0.144928
Northeast,0.760000,0.240000
Northwest,0.469697,0.530303
South,0.676856,0.323144
Southeast,0.934641,0.065359
Southwest,0.489130,0.510870
West,0.608295,0.391705
West North Central,0.411765,0.588235


In [75]:
pivot_copy.plot(kind='barh', barmode='group', title='Distribution of Missing Data Across all Climate Regions')


In [76]:
obs_tvd = pivoted.diff(axis=1).iloc[:,-1].abs().sum()/ 2
obs_tvd

np.float64(322.5)

In [77]:
n_repetitions = 1000
shuffled = data.copy()
tvds = []
for _ in range(n_repetitions):

    # Shuffling genders. 
    # Note that we are assigning back to the same DataFrame for performance reasons; 
    # see https://dsc80.com/resources/lectures/lec07/lec07-fast-permutation-tests.html.
    shuffled['shuffled'] = np.random.permutation(shuffled['CLIMATE.REGION'])

    # Computing and storing the TVD.
    shuffled['is_missing'] = shuffled['CUSTOMERS.AFFECTED'].isna()

    pivoted = shuffled.pivot_table(index='shuffled', 
                                            columns='is_missing', 
                                            aggfunc='size',
                                            fill_value=0)

    tvd = pivoted.diff(axis=1).iloc[:, -1].abs().sum() / 2
    tvds.append(tvd)

In [78]:
fig = px.histogram(x=tvds, histnorm = 'probability', labels={'x': 'TVD'}, title='Empirical Distribution of TVD')
fig.add_vline(
    x=obs_tvd,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Observed TVD = {obs_tvd}",
)
fig.show()

In [79]:
(np.array(tvds) >= obs_tvd).mean()

np.float64(0.553)

The observed TVD is 334.0, resulting in the p-value of 0.0, which is less than our significance level of 0.05. We reject the null hypothesis and conclude that the missingness of CUSTOMERS.AFFECTED is dependent on the column CLIMATE.CATEGORY

## Step 4: Hypothesis Testing


We will be performing a permutation test to determine if the number of customers affected by power outages in the South region is greater than the number of customers affected by power outages in the Northeast region. We will be using the columns of CUSTOMERS.AFFECTED and CLIMATE.REGION. Within the CLIMATE.REGION columns, we will be focusing on the values of South and Northeast as those are what we will be observing in the permutation test.

Null Hypothesis: On average, the number of customers affected by power outages in South region is the same as the number of customers affected by power outage in Northeast region.

Alternate Hypothesis: On average, the number of customers affected by power outages in South region is greater than the number of customers affected by power outage in Northeast region.

Test statistic: difference in mean, (mean number of customers affect in South region) - (mean number of customers afected in Northeast region)

In [80]:
#calculate absolute difference of observed data
enc_stats = data[data['CLIMATE.REGION'] == 'South']['CUSTOMERS.AFFECTED']
swc_stats = data[data['CLIMATE.REGION'] == 'Northeast']['CUSTOMERS.AFFECTED']

enc_mean = enc_stats.mean()
swc_mean = swc_stats.mean()

obs_stats = enc_mean - swc_mean

#permutation test
n_permutations = 10000
permutated_stats = []

data_copy = data.copy()
for i in range(n_permutations):
    shuffled_regions = np.random.permutation(data_copy['CLIMATE.REGION'])
    data_copy['shuffled_regions'] = shuffled_regions

    perm_enc = data_copy[data_copy['shuffled_regions'] == 'South']['CUSTOMERS.AFFECTED']
    perm_swc = data_copy[data_copy['shuffled_regions'] == 'Northeast']['CUSTOMERS.AFFECTED']

    perm_enc_mean = perm_enc.mean()
    perm_swc_mean = perm_swc.mean()

    perm_stat = perm_enc_mean - perm_swc_mean

    permutated_stats.append(perm_stat)


p_value = np.mean(np.array(permutated_stats) >= obs_stats)
p_value

np.float64(0.0196)

0.05 signiicance level
>> reject null hypothesis

In [81]:
fig = px.histogram(
    permutated_stats,
    nbins=30,
    title="Permutation Test: Empirical Distribution of Differences in Means",
    labels={'value': 'Difference in Means'},
    histnorm='probability'
)
fig.add_vline(
    x=obs_stats,
    line_dash="dash",
    line_color="red",
    annotation_text=f"Observed Stat = {obs_stats}"
)
fig.show()

We performed 10,000 simulations to create our empirical distribution under the null hypothesis with our chosen test statistic. Our observed statistic of 61540 landed us at the p-value of 0.0204. With the significance level of 0.05, we reject the null hypothesis in favor for the alternative, indicating that the numbers of customers affected in South region is greater than the number of customers affected in the Northeast region. We conclude that outages in the South tend to be more severe.

## Step 5: Framing a Prediction Problem

Our prediction problem is a binary classification task to predict ‘CLIMATE.REGION’, identifying whether a major outage occurred in the South or Northeast. This helps address regional differences in outage causes, so that we can allocate our resources to helping the regions who are impacted more severely by major power outages.

We use only features available at the time of prediction including 'CLIMATE.CATEGORY', 'ANOMALY.LEVEL', 'YEAR', 'RES.PERCEN', 'CLIMATE.REGION'. These information about the overall characteristics of the regions and the year of which the outage took place will help us predict which region a power outage would happen in the South or Northeast. The F1-score was chosen as our evaluation metric which combines the measurements of precision and recall, ensuring more accurate predictions in our classification model, which is a better metric than accurancy for classification models as it takes into account of class imbalance.

## Step 6: Baseline Model

### Getting Necessary Data
Columns: `'CLIMATE.CATEGORY'`, `'ANOMALY.LEVEL'`, `'YEAR'`, `'RES.PERCEN'`

What we are predicting: `'CLIMATE.REGION'`

In [82]:
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [83]:
south_ne = data[(data['CLIMATE.REGION'] == 'Northeast') | (data['CLIMATE.REGION'] == 'South')]
south_ne

,YEAR,MONTH,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,OUTAGE.DURATION,CUSTOMERS.AFFECTED,RES.PERCEN,ANOMALY.LEVEL,TOTAL.REALGSP,DEMAND.LOSS.MW,customers_affected_ranges
168,2015,7,TRE,South,warm,system operability disruption,373,30000,41.399483,1.2,1488049,350,"(0, 1M]"
169,2011,2,TRE,South,cold,severe weather,1203,NaN,41.451897,-1,1246833,NaN,NaN
170,2011,2,TRE,South,cold,system operability disruption,868,86013,41.451897,-1,1246833,400,"(0, 1M]"
171,2016,7,TRE,South,normal,severe weather,1455,52000,NaN,-0.3,1481866,NaN,"(0, 1M]"
172,2012,8,WECC,South,normal,intentional attack,206,3314,43.165452,0.3,1314004,12,"(0, 1M]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510,2007,12,MRO,South,cold,severe weather,13650,95000,34.999,-1.3,125844,500,"(0, 1M]"
1511,2015,12,SPP,South,warm,intentional attack,90,0,33.643859,2.3,135967,0,"(0, 1M]"
1512,2011,6,SPP,South,normal,public appeal,913,NaN,38.003532,-0.3,131149,NaN,NaN
1513,2015,6,SPP,South,warm,severe weather,NaN,110000,36.565748,1,135967,NaN,"(0, 1M]"


In [84]:
#Getting the needed Columns for the Needed Features
nec_cols = ['CLIMATE.CATEGORY', 'ANOMALY.LEVEL', 'YEAR', 'RES.PERCEN', 'CLIMATE.REGION']

south_ne_nec = pd.DataFrame()

for col in nec_cols:
    south_ne_nec[col] = south_ne[col]
south_ne_nec['CLIMATE.REGION'] = south_ne_nec['CLIMATE.REGION'].replace({'Northeast': 0, 'South': 1})
south_ne_nec

/var/folders/r2/plrh1xlx4_b6wmq2hw3_92dw0000gn/T/ipykernel_84582/1207825865.py:8: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,CLIMATE.CATEGORY,ANOMALY.LEVEL,YEAR,RES.PERCEN,CLIMATE.REGION
168,warm,1.2,2015,41.399483,1
169,cold,-1,2011,41.451897,1
170,cold,-1,2011,41.451897,1
171,normal,-0.3,2016,NaN,1
172,normal,0.3,2012,43.165452,1
...,...,...,...,...,...
1510,cold,-1.3,2007,34.999,1
1511,warm,2.3,2015,33.643859,1
1512,normal,-0.3,2011,38.003532,1
1513,warm,1,2015,36.565748,1


- X: all the features 
- y: actual outcomes

In [85]:
X = south_ne_nec.drop(columns='CLIMATE.REGION')
y = south_ne_nec['CLIMATE.REGION']

### Build the Pipeline

- Just OHE `'CLIMATE.CATEGORY'`
- leave everything else as is
- use `RandomForestClassifier` for the model



In [86]:
col_trans = ColumnTransformer(
    transformers=[
        ('ohe climate categories', OneHotEncoder(handle_unknown='ignore'), ['CLIMATE.CATEGORY']),
    ],
    remainder='passthrough',
    force_int_remainder_cols=False
)

pl = Pipeline([
    ('preproc', col_trans),
    ('random forest', RandomForestClassifier(random_state=42))
])

### Evaluate for the Accuracy and f1 Score
1. Split the data into training and testing sets
2. Fit our pipeline
3. Calculate $R^2$
4. Calculate f1 Score
5. Repeat Steps 1 to 4 100 times
6. Take the average of the $R^2$'s and f1 socres

In [87]:
r_2s = []
f1_scores = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
    
    pl.fit(X_train, y_train)

    r_2 = pl.score(X_test, y_test)
    r_2s.append(r_2)

    y_pred = pl.predict(X_test)
    f1_score_val = f1_score(y_test, y_pred)
    f1_scores.append(f1_score_val)

#### Value of $R^2$

In [88]:
avg_r_2 = np.array(r_2s).mean()
avg_r_2

np.float64(0.7994252873563218)

#### Value of F1 Score

In [89]:
avg_f1_score = np.array(f1_scores).mean()
avg_f1_score

np.float64(0.7408488275588584)

Our model was built to predict the ‘CLIMATE.REGION’ (binary: Northeast or South) using the features of 'ANOMALY.LEVEL' (quantitative), 'YEAR' (ordinal), 'RES.PERCEN' (quantitative), and 'CLIMATE.CATEGORY' (nominal). The 'CLIMATE.CATEGORY' feature was one-hot encoded, while quantitative features were passed directly into the model without scaling. We used ANOMALY.LEVEL and CLIMATE.CATEGORY to provide us with information of the climate in the different climate regions, YEAR to account for changes over time, and RES.PERCEN to offer economical statistics about the two regions.The target, 'CLIMATE.REGION', was binarized (0 for Northeast, 1 for South).

The pipeline combined a ColumnTransformer for preprocessing and the Random Forest Classifier for predictions. Because of the amount of variation in the evaluation metrics, we ran the model 100 times with different splits training and testing sets and calculated the averages. Over the course of 100 simulations, the $R^2$ is 0.802, while the F-1 Score is 0.744.

Overall, the model performed well as a baseline, with strong metrics and effective handling of both categorical and numerical features. While missing values in 'RES.PERCEN' could be addressed to enhance performance, the current implementation provides a solid foundation for predicting climate regions.

## Step 7: Final Model



### Function for Standardizing By Group

In [90]:
from sklearn.base import BaseEstimator, TransformerMixin

class StdScalerByGroup(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        # X might not be a pandas DataFrame (e.g. a np.array)
        df = pd.DataFrame(X)
        # display(df)

        # Compute and store the means/standard-deviations for each column (e.g. 'c1' and 'c2'), 
        # for each group (e.g. 'A', 'B', 'C').  
        # (Our solution uses a dictionary)
        self.grps_ = {}
        groups = df.iloc[:, 0]
        group_name = df.columns[0]
        all_groups = groups.unique()

        for group in all_groups:
            group_data = df[df[group_name] == group]
            group_data.drop(columns=group_name, inplace=True)
            # display(group_data)
            agg_data = group_data.agg(['mean', 'std'])
            self.grps_[group] = agg_data

        return self

    def transform(self, X, y=None):

        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        
        # Hint: Define a helper function here!
        df = pd.DataFrame(X)
        transformed = df.copy()

        groups = df.iloc[:, 0]
        group_name = df.columns[0]
        all_cols = df.columns[1:]

        def standardize(group_data, group):
            # display(self.grps_)

            for column in all_cols:
                mean = self.grps_[group][column].loc['mean']
                std = self.grps_[group][column].loc['std']
                # print(f"Column: {column}, Mean: {mean}, Std: {std}, group: {group}")
                group_data[column] = (group_data[column] - mean)/std
            return group_data

        transformed = transformed.groupby(group_name, group_keys=False).apply(
            lambda group: standardize(group, group.name)
        )
        transformed.drop(columns=group_name, inplace=True)
        return transformed

### Getting necessary data and feature

In [91]:
nec_cols = ['CLIMATE.CATEGORY', 'ANOMALY.LEVEL', 'YEAR', 'RES.PERCEN', 'MONTH', 'TOTAL.REALGSP', 'CLIMATE.REGION']

final_df = pd.DataFrame()

for col in nec_cols:
    final_df[col] = south_ne[col]
final_df['CLIMATE.REGION'] = final_df['CLIMATE.REGION'].replace({'Northeast': 0, 'South': 1})
final_df

/var/folders/r2/plrh1xlx4_b6wmq2hw3_92dw0000gn/T/ipykernel_84582/3203821078.py:7: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,CLIMATE.CATEGORY,ANOMALY.LEVEL,YEAR,RES.PERCEN,MONTH,TOTAL.REALGSP,CLIMATE.REGION
168,warm,1.2,2015,41.399483,7,1488049,1
169,cold,-1,2011,41.451897,2,1246833,1
170,cold,-1,2011,41.451897,2,1246833,1
171,normal,-0.3,2016,NaN,7,1481866,1
172,normal,0.3,2012,43.165452,8,1314004,1
...,...,...,...,...,...,...,...
1510,cold,-1.3,2007,34.999,12,125844,1
1511,warm,2.3,2015,33.643859,12,135967,1
1512,normal,-0.3,2011,38.003532,6,131149,1
1513,warm,1,2015,36.565748,6,135967,1


In [92]:
X = final_df.drop(columns='CLIMATE.REGION')
y = final_df['CLIMATE.REGION']

### Helper Functions of Probabilistic Imputation

In [93]:
def impute(col):
    #Probabilistic Imputation of Month
    n = col.isna().sum()
    probabilities = col.dropna().value_counts()/col.dropna().value_counts().sum()
    fill = np.random.choice(probabilities.index, n, p=probabilities.values)
    col[col.isna()] = fill
    return col

### Function for Transformation

In [94]:
def month_year_fe(df):
    #Probabilistic Imputation of Month
    months = impute(df['MONTH'])

    #Getting number of months since Jan. 2000
    num_months_year = (df['YEAR'] - 2000) * 12
    tot_months = num_months_year + (months - 1)
    df['months_from2000'] = tot_months

    #get seasons
    def seasons(month):
        if month in [12, 1, 2]:
            return 'winter'
        elif month in [3, 4, 5]:
            return 'spring'
        elif month in [6, 7, 8]:
            return 'summer'
        else:
            return 'autumn'
        
    df['seasons'] = months.apply(seasons)

    new_df = df.drop(columns=['YEAR', 'MONTH']).copy()
    return new_df

### Building the Pipeline
What is in the Pipeline?
- **FunctionTransformer with month_func as the function:** creates two new features
    - `months_from2000`: gets the number of months after January 2000 in that specific month is when the outage occured
    - `seasons`: the season in which the outage occured
- **ColumnTransformer:** does OneHotEncoding on `seasons` and `CLIMATE.CATEGORY`
- **RandomForestClassifier:** our classifier model

In [95]:

month_func = FunctionTransformer(month_year_fe)
final_tree = RandomForestClassifier(max_depth=12, n_estimators=40, random_state=42)

# col_anomaly = ColumnTransformer(
#     transformers=[
#         ('std by group anomaly level', StdScalerByGroup(), ['CLIMATE.CATEGORY','ANOMALY.LEVEL'])
#     ],
#     remainder='passthrough'
# )
pl_months = Pipeline([
    # ('std by group anomaly levels', col_anomaly),
    ('total months function', month_func),
    ('col transform', ColumnTransformer(
        transformers=[
            ('ohe seasons', OneHotEncoder(drop='first', handle_unknown='ignore'), ['seasons', 'CLIMATE.CATEGORY']),
        ],
        remainder='passthrough',
        force_int_remainder_cols=False
    )),
    ('randomtreeclassifier', final_tree),
])

### Finding the Optimal Hyperparameters
Currently commented Out because it might error from inputting the correct hyperparameters above

Optimal Hyperparameters:
- `max_depth` = 12
- `n_estimators` = 40

In [96]:
# from sklearn.model_selection import GridSearchCV
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
# pl_months.fit(X_train, y_train)
# X_trans= pl_months.transform(X_train)
# test_tree = RandomForestClassifier(random_state=42)

# hyperparameters = {
#     'max_depth': [12], 
#     'n_estimators': np.arange(20, 50, 2),
#     # 'criterons': ['gini', 'entropy']
# }

# searcher = GridSearchCV(test_tree, hyperparameters, cv=5)
# searcher.fit(X_trans, y_train)
# print(searcher.best_params_)

### Evaluate for the Accuracy and f1 Score

In [97]:
final_r_2s = []
final_f1_scores = []

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)
    
    pl_months.fit(X_train, y_train)

    r_2 = pl_months.score(X_test, y_test)
    final_r_2s.append(r_2)

    y_pred = pl_months.predict(X_test)
    f1_score_val = f1_score(y_test, y_pred)
    final_f1_scores.append(f1_score_val)


#### Value of $R^2$

In [98]:
final_avg_r_2 = np.array(final_r_2s).mean()
final_avg_r_2

np.float64(0.9556321839080459)

#### Value of F1 Score

In [99]:
final_avg_f1_score = np.array(final_f1_scores).mean()
final_avg_f1_score

np.float64(0.9426139686249558)

In our final model, we used the following features 'CLIMATE.CATEGORY', 'ANOMALY.LEVEL', 'YEAR', 'RES.PERCEN', 'MONTH', 'TOTAL.REALGSP', 'CLIMATE.REGION' from the data set as well as created two other features of months_from_2000 and seasons.

We used YEAR and MONTH to calculate the number of months that passed since January 2000, hence months_from_2000 and used MONTH to categorize the rows based on season: winter, spring, summer, autumn. These two new features were created via a FunctionTransformer. We also added the quantative feature of TOTAL.REALGSP because it is a better feature in estimating the economic characteristics rather than just RES.PERCEN, which only takes account of electricity consumption in residential areas.

We then used a ColumnTransformer to OneHotEncode the nominal columns of seasons and CLIMATE.CATEOGORY. With the OneHotEncoding, we handled unknown values that may be absent in the training set but exist in the test set by ignoring them and dropping the first OHE feature to avoid multicollinearity. The remaining features are passthrough and remain unchanged. These were all passed into a pipeline with the RandomTreeClassifier being our main classifier model. We chose to use a Random Forest Classifier because it works well with mixed data types and can handle complex relationships between features as well as taking the average of multiple DecisionTrees.

To find the optimal hyperparameters for our RandomTreeClassifier, we used GridSearchCV under the assumption of random_state=42 to have more consistent results, giving us the results of

max_depth = 12
n_estimators = 40
Overall, we noticed an improvement in our final model with an average $R^2$ across 100 simulations (similar to before) of 0.952 and an F-1 Score of 0.939.

## Step 8: Fairness Analysis

For the Fairness Analysis Model, we computed the difference in the distribution of the f1_score between the ‘is_winter,’ counting months [12, 1, 2] as ‘is_winter’ against the ‘not_winter’ group, in which we created a new is_winter column as a tranformation from the ‘MONTH’ column. We chose this analysis and thinks it supports our main argument ‘where are the causes of major outages?’ because while we we’re making a new column for each ‘Season’ we realized that the outages in the month winter are disporportion to the other seasons. Therefore, we wanted to investigate if our model is fair based of the data.

Null Hypothesis: There are no differences between the distribution of the f1_scores of ‘is_winter’ and not ‘is_winter’. Alternate Hypothesis: There is a differences between the distribution of the f1_scores of ‘is_winter’ and not ‘is_winter’.

In [100]:
#Does my model perform worse 
from sklearn import metrics
metrics.precision_score(y_test, y_pred)

np.float64(0.9722222222222222)

In [101]:
len(y_pred)

87

In [102]:
X

,CLIMATE.CATEGORY,ANOMALY.LEVEL,YEAR,RES.PERCEN,MONTH,TOTAL.REALGSP
168,warm,1.2,2015,41.399483,7,1488049
169,cold,-1,2011,41.451897,2,1246833
170,cold,-1,2011,41.451897,2,1246833
171,normal,-0.3,2016,NaN,7,1481866
172,normal,0.3,2012,43.165452,8,1314004
...,...,...,...,...,...,...
1510,cold,-1.3,2007,34.999,12,125844
1511,warm,2.3,2015,33.643859,12,135967
1512,normal,-0.3,2011,38.003532,6,131149
1513,warm,1,2015,36.565748,6,135967


In [103]:
results = X_test
# results['age_bracket'] = results['age'].apply(lambda x: 5 * (x // 5 + 1))
results['prediction'] = y_pred
results['tag'] = y_test

results

,CLIMATE.CATEGORY,ANOMALY.LEVEL,YEAR,RES.PERCEN,MONTH,TOTAL.REALGSP,months_from2000,seasons,prediction,tag
1451,cold,-1.3,2007,40.648737,12,51518,95,winter,0,0
549,normal,-0.1,2012,40.630826,6,317123,149,summer,0,0
1396,normal,-0.3,2011,44.452325,7,226534,138,summer,0,0
936,cold,-0.7,2008,35.105046,12,1120512,107,winter,0,0
1460,cold,-0.7,2008,41.85405,12,51162,107,winter,0,0
...,...,...,...,...,...,...,...,...,...,...
346,normal,-0.1,2014,31.878841,5,94766,172,spring,1,1
1395,normal,-0.2,2013,42.663058,4,226209,159,spring,0,0
613,normal,0.1,2012,39.185153,7,595700,150,summer,0,0
1456,warm,2.3,2015,40.761924,12,51007,191,winter,1,0


In [104]:
def compute_f1(group):
  
    true_labels = group['tag']
    predicted_labels = group['prediction']
    return f1_score(true_labels, predicted_labels, zero_division=0) 

In [105]:
winter_months = [12, 1, 2]

results['is_winter'] = results['MONTH'].apply(lambda x: 'Winter' if x in winter_months else 'Non-Winter')

group_winter = results[results['is_winter'] == 'Winter']
group_winter

,CLIMATE.CATEGORY,ANOMALY.LEVEL,YEAR,RES.PERCEN,MONTH,TOTAL.REALGSP,months_from2000,seasons,prediction,tag,is_winter
1451,cold,-1.3,2007,40.648737,12,51518,95,winter,0,0,Winter
936,cold,-0.7,2008,35.105046,12,1120512,107,winter,0,0,Winter
1460,cold,-0.7,2008,41.85405,12,51162,107,winter,0,0,Winter
1411,normal,-0.2,2002,43.732486,1,123183,24,winter,1,1,Winter
1449,normal,-0.4,2013,39.114619,2,411601,157,winter,0,0,Winter
275,cold,-0.5,2014,40.550511,1,1421759,168,winter,1,1,Winter
205,cold,-1,2011,41.451897,2,1246833,133,winter,1,1,Winter
868,cold,-0.7,2012,43.151817,1,57146,144,winter,0,0,Winter
1464,normal,-0.3,2013,43.480206,12,49755,167,winter,0,0,Winter
907,cold,-0.7,2012,43.151817,1,57146,144,winter,0,0,Winter


In [106]:
group_non_winter = results[results['is_winter'] == 'Non-Winter']


macro_f1_winter = f1_score(group_winter['tag'], group_winter['prediction'], average='macro')
macro_f1_non_winter = f1_score(group_non_winter['tag'], group_non_winter['prediction'], average='macro')

observed_diff = macro_f1_winter - macro_f1_non_winter

f1_differences = [] 

for _ in range(500):
    results['shuffled_is_winter'] = np.random.permutation(results['is_winter'])

    shuffled_winter = results[results['shuffled_is_winter'] == 'Winter']
    shuffled_non_winter = results[results['shuffled_is_winter'] == 'Non-Winter']

    #do f1_score separately for these two 
    macro_f1_shuffled_winter = f1_score(shuffled_winter['tag'], shuffled_winter['prediction'], average='macro')
    macro_f1_shuffled_non_winter = f1_score(shuffled_non_winter['tag'], shuffled_non_winter['prediction'], average='macro')

    f1_differences.append(macro_f1_shuffled_winter - macro_f1_shuffled_non_winter)
    
# print(macro_f1_shuffled_winter)
print(f1_differences)
p_value = np.mean([abs(diff) >= abs(observed_diff) for diff in f1_differences])

print(f"P-value: {p_value}")

[np.float64(0.03273809523809523), np.float64(0.03173076923076923), np.float64(0.03219512195121954), np.float64(-0.03382734912146679), np.float64(0.03173076923076923), np.float64(0.03101503759398494), np.float64(-0.03382734912146679), np.float64(0.030555555555555558), np.float64(-0.09722222222222232), np.float64(0.03101503759398494), np.float64(0.03173076923076923), np.float64(0.03219512195121954), np.float64(-0.03164228654424739), np.float64(0.03101503759398494), np.float64(-0.0403790264322037), np.float64(-0.03164228654424739), np.float64(-0.03382734912146679), np.float64(-0.0403790264322037), np.float64(0.031339031339031376), np.float64(0.030330882352941124), np.float64(-0.036413255360623786), np.float64(0.030555555555555558), np.float64(-0.09545454545454546), np.float64(0.031339031339031376), np.float64(0.03173076923076923), np.float64(0.03219512195121954), np.float64(-0.03382734912146679), np.float64(0.03101503759398494), np.float64(-0.03382734912146679), np.float64(-0.036413255360

In [107]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame({'F1 Differences': f1_differences})

fig = px.histogram(
    df,
    x='F1 Differences',
    nbins=20,
    histnorm='probability',
    title='Distribution of the F1 Difference for Longer vs Shorter Outages',
    labels={'x': 'F1 Difference Longer vs Shorter Outages', 'y': 'Probability'}
)

fig.add_vline(x=observed_diff, line_color='red', annotation_text=f'Observed Diff = {observed_diff:.2f}', annotation_position='top left')
fig.show()
#in our training dataset, we had the concern that more of our data is in the winter region, making our model unfair 
#reject the null, showing that there ARE bias towards the winter regions, possibly that there are more winter region data

We utilized permutation test to test ‘differences in f1_score distribution’ between the ‘is_winter’ data and not ‘not_winter’ data. We got a p_value of: 0.13, which is greater than the observed statistic of 0.05, so we reject our null hypothesis and conclude that there is a differences between the distribution of the f1_scores of ‘is_winter’ and ‘not_winter’.